# <center> Bài tập Thực hành môn Khai phá Dữ liệu </center>

### **Họ và tên:** Huỳnh Nguyễn Thế Dân
### **MSSV:** 21110256
### **Lớp:** 21TTH1

##  1. Sử dụng thư viện

In [1]:
import numpy as np
import pandas as pd
from pyECLAT import ECLAT # type: ignore

In [12]:
dataframe = pd.read_csv('data.csv', header= None)

display(dataframe)

,0,1,2,3,4,5
0,Wine,Chips,Bread,Butter,Milk,Apple
1,Wine,NaN,Bread,Butter,Milk,NaN
2,NaN,NaN,Bread,Butter,Milk,NaN
3,NaN,Chips,NaN,NaN,NaN,Apple
4,Wine,Chips,Bread,Butter,Milk,Apple
5,Wine,Chips,NaN,NaN,Milk,NaN
6,Wine,Chips,Bread,Butter,NaN,Apple
7,Wine,Chips,NaN,NaN,Milk,NaN
8,Wine,NaN,Bread,NaN,NaN,Apple
9,Wine,NaN,Bread,Butter,Milk,NaN


In [19]:
eclat_instance = ECLAT(data= dataframe, verbose= True)

display(eclat_instance.df_bin)

100%|██████████| 6/6 [00:00<00:00, 2249.16it/s]


,Wine,Milk,Chips,Bread,Apple,Butter
0,1,1,1,1,1,1
1,1,1,0,1,0,1
2,0,1,0,1,0,1
3,0,0,1,0,1,0
4,1,1,1,1,1,1
5,1,1,1,0,0,0
6,1,0,1,1,1,1
7,1,1,1,0,0,0
8,1,0,0,1,1,0
9,1,1,0,1,0,1


In [23]:
# Count items in each row
items_per_transaction = eclat_instance.df_bin.astype(int).sum(axis = 1)

# The item should appear at least at 5% transactions
min_support = 0.5

# Start from transactions containing at least 2 items
min_combination = 1

# Up to maximum items per transaction
max_combination = max(items_per_transaction)
rule_indices, rule_supports = eclat_instance.fit(min_support= min_support, min_combination= min_combination, max_combination= max_combination, separator= ' & ', verbose= True)

result = pd.DataFrame(rule_supports.items(), columns=['Item', 'Support'])
result.sort_values(by= ['Support'], ascending= False)
display(result)

Combination 1 by 1


6it [00:00, 332.83it/s]


Combination 2 by 2


15it [00:00, 424.69it/s]


Combination 3 by 3


...

Combination 5 by 5


6it [00:00, 468.86it/s]


Combination 6 by 6


1it [00:00, 333.17it/s]


,Item,Support
0,Wine,0.727273
1,Milk,0.772727
2,Chips,0.636364
3,Bread,0.727273
4,Apple,0.681818
5,Butter,0.681818
6,Wine & Milk,0.636364
7,Wine & Bread,0.590909
8,Wine & Apple,0.500000
9,Wine & Butter,0.500000


##  2. Cài đặt lại thuật toán Vertical Apriori

In [66]:
from itertools import combinations

def VerticalApriori(df_bin, min_support):
    # Bước 1: Tạo dạng biểu diễn dọc của dữ liệu
    itemsets = {}
    num_transactions = len(df_bin)
    
    for item in df_bin.columns:
        transactions = set(df_bin.index[df_bin[item] == 1].tolist())
        itemsets[frozenset([item])] = transactions
    
    # Bước 2: Lọc các mục đơn thường xuyên
    frequent_itemsets = {}
    for itemset, transactions in itemsets.items():
        if len(transactions) / num_transactions >= min_support:
            frequent_itemsets[itemset] = transactions
    
    # Bước 3: Tạo các tập hợp mục thường xuyên lớn hơn từ các mục thường xuyên hiện tại
    current_itemsets = frequent_itemsets.copy()
    k = 2
    while current_itemsets:
        # Tạo tập hợp mục mới kết hợp từ tập hợp mục hiện tại
        new_itemsets = {}
        keys = list(current_itemsets.keys())
        for c in combinations(keys, 2):
            combined_itemset = c[0] | c[1]
            if len(combined_itemset) == k:
                combined_transactions = current_itemsets[c[0]].intersection(current_itemsets[c[1]])
                if len(combined_transactions) / num_transactions >= min_support:
                    new_itemsets[combined_itemset] = combined_transactions
        
        # Cập nhật tập hợp mục thường xuyên
        frequent_itemsets.update(new_itemsets)
        current_itemsets = new_itemsets
        k += 1

    return frequent_itemsets


In [63]:
# Sử dụng cài đặt
df_bin = eclat_instance.df_bin.astype(int)  # Giả sử df_bin đã được định nghĩa
min_support = 0.5  # Giả sử mức hỗ trợ tối thiểu là 5%
frequent_itemsets = VerticalApriori(df_bin, min_support)

In [65]:
# Xuất kết quả
# Bước 1: Tạo data
data = []
for itemset, transactions in frequent_itemsets.items():
    support = len(transactions) / len(df_bin)  # Tính số lượng giao dịch mà tập hợp mục xuất hiện
    itemset_formatted = ' & '.join(itemset)  # Định dạng itemset thành chuỗi để hiển thị tốt hơn
    data.append([itemset_formatted, support])

# Bước 2: Tạo DataFrame
result_df = pd.DataFrame(data, columns=['Itemset', 'Support'])

# Bước 3: Hiển thị hoặc Lưu DataFrame
display(result_df)

,Itemset,Support
0,Wine,0.727273
1,Milk,0.772727
2,Chips,0.636364
3,Bread,0.727273
4,Apple,0.681818
5,Butter,0.681818
6,Wine & Milk,0.636364
7,Wine & Bread,0.590909
8,Wine & Apple,0.500000
9,Butter & Wine,0.500000
